### TFIDF -> Random Forest Regressor

#### Nessecary Imports

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np

#### Load in data / Separate out/ drop numerical data, isolate text data.

In [18]:
train_df = pd.read_csv('checkpoint_train_v1.csv', encoding = 'utf-8')
test_df = pd.read_csv('checkpoint_test_v1.csv', encoding = 'utf-8')

In [6]:
text_columns = ['overview', 'tagline', 'title', 'all_keywords', 'part_of_collection']
numerical_columns = ['userId', 'movieId', 'release_date', 'budget', 'popularity', 'revenue', 
                     'runtime', 'vote_average', 'vote_count', 'old', 
                     'Science Fiction', 'Crime', 'Thriller', 'Drama', 
                     'Horror', 'Romance', 'TV Movie', 'Action', 
                     'Adventure', 'War', 'Documentary', 'Family', 
                     'Fantasy', 'Music', 'Animation', 'Mystery', 
                     'Comedy', 'Western', 'Foreign', 'History']

#### Create combined text column

In [8]:
train_df = train_df.fillna('').astype(str)
test_df = test_df.fillna('').astype(str)

In [9]:
train_df['corpus'] = train_df['overview'] + "_" + train_df['tagline'] + "_" + train_df['title'] + "_" + train_df['all_keywords'] + "_" + train_df['part_of_collection']
test_df['corpus'] = test_df['overview'] + "_" + test_df['tagline'] + "_" + test_df['title'] + "_" + test_df['all_keywords'] + "_" + test_df['part_of_collection']

#### Initialize Vectorizer, fit to text column, create tfidf matricies

In [10]:
vectorizer = TfidfVectorizer()
vectorizer.fit(train_df['corpus'])
train_tfidf_matrix = vectorizer.transform(train_df['corpus'])
test_tfidf_matrix = vectorizer.transform(test_df['corpus'])

#### Create train/ test sets

In [11]:
y_train = train_df['rating']
X_train = train_tfidf_matrix

X_test = test_tfidf_matrix

#### Initialize/ Train Random Forest Model

In [12]:
rf_parameters = {
    'random_state': 150,
    'n_estimators': 500,
    'max_depth': 10
}

In [13]:
RF_regressor = RandomForestRegressor(**rf_parameters)
RF_regressor.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, n_estimators=500, random_state=150)

#### Save / Submit Results

In [20]:
test_df['rating'] = (RF_regressor.predict(X_test))
test_df['movieId'] = test_df['movieId'].astype(int).astype(str)
test_df['userId'] = test_df['userId'].astype(int).astype(str)
test_df['userId_movieId'] = test_df['userId'] + '_' + test_df['movieId']
cols = ['userId_movieId', 'rating']
test_df = test_df[cols]

In [21]:
test_df.head(1)

,userId_movieId,rating
0,469_2124,0.674123


In [22]:
test_df.to_csv('submission.csv', index = False)

Model RMSE: 0.2055

### Idea for a next step: Do some hyperparam tuning / add the other numerical and categorical features and do apples to apples comparisons with this model